<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe30338-2021](https://jckantor.github.io/cbe30338-2021);
content is available [on Github](https://github.com/jckantor/cbe30338-2021.git).*


<!--NAVIGATION-->
< [5.2 Linear Blending Problems](https://jckantor.github.io/cbe30338-2021/05.02-Linear-Blending-Problem.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.4 Gasoline Blending](https://jckantor.github.io/cbe30338-2021/05.04-Gasoline-Blending.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.03-Homework_5.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.03-Homework_5.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 5.3 Homework Assignment 4

Homework assigment 4 is intended to develop your skills in creating linear programming models. The lecture notes (notebook 5.2) will provide with a starting point for the following exercises. The problem data has been changed to provide a more meaningful example for sensitivity analysis.


You have been placed in charge of a metals reuse operation. A collection of raw materials are available with key parameters given in the following table.

| material | % carbon (C) | % copper (Cu) | % manganese (Mn) | amount (kg) | cost (\$/kg) | type |
| :---: | :---: | :---: | :---: | :---: | :---: | :---
| A | 2.5 | 0.0 | 1.3 | 4000 | 1.20 | iron alloy
| B | 3.0 | 0.0 | 0.8 | 3000 | 1.50 | iron alloy
| C | 0.0 | 0.3 | 0.0 | 6000 | 0.90 | iron alloy
| D | 0.0 | 90.0 | 0.0 | 5000 | 1.30 | copper alloy
| E | 0.0 | 96.0 | 4.0 | 2000 | 1.45 | copper alloy
| F | 0.0 | 0.4 | 1.2 | 3000 | 1.20 | aluminum alloy
| G | 0.0 | 0.6 | 0.0 | 2500 | 1.00 | aluminum alloy

A customer has requested 5000 kg of mix satisfying these  specifications:

| Component | min % | max % |
| :---: | :---: | :---: |
| C | 2.0 | 3.0 |
| Cu | 0.4 | 0.6 |
| Mn | 1.2 | 1.65 |

For convenience, the raw material data has been organized as a nested dictionaries.

In [ ]:
data = {
    "A": {"C": 2.5, "Cu": 0.0, "Mn": 1.3, "amount": 4000, "cost": 1.20},
    "B": {"C": 3.0, "Cu": 0.0, "Mn": 0.8, "amount": 3000, "cost": 1.50},
    "C": {"C": 0.0, "Cu": 0.3, "Mn": 0.0, "amount": 6000, "cost": 0.90},
    "D": {"C": 0.0, "Cu": 90.0, "Mn": 0.0, "amount": 5000, "cost": 1.30},
    "E": {"C": 0.0, "Cu": 96.0, "Mn": 4.0, "amount": 2000, "cost": 1.45},
    "F": {"C": 0.0, "Cu": 0.4, "Mn": 1.2, "amount": 3000, "cost": 1.20},
    "G": {"C": 0.0, "Cu": 0.6, "Mn": 0.0, "amount": 2500, "cost": 1.00},
}

## 5.3.1 Exercise 1. Getting Started




Considering just the constraint on carbon content, adapt the `brew_blend` function from notebook 5.2 to find a minimum cost blend raw materials that yield 5000 kg of product with a carbon content between 2.0 and 3.0%. For this first exercise you can ignore the bounds on the available amount of each raw material.

* change the name of the function to `metal_blend`.
* `metal_blend` should accept arguments including
    * `data`, 
    * the required mass of final product, and
    *  the acceptable range of carbon content specified as a pair of numbers of  `[lower_bound, upper_bound]`.
* `metal_blend` should return the minimum cost, and the amounts of each raw material to include in the blend.

Demonstrate use of `metal_blend` to compute the required blend. From the results of the calculation to report

* the cost of the blend
* the total mass of the blend
* the mass of each raw material used in the blend
* the composition of the blend with regard to the species carbon, copper and managanese.

You may find it convenient to write a function for this purpose that can be used in the following exercise.



### 5.3.1.1 Solution

In [ ]:
import numpy as np
import cvxpy as cp

def metal_blend(mass_total, Carbon_bounds, data):
    
    # extract bounds on carbon content
    C12_lower = Carbon_bounds[0]
    C12_upper = Carbon_bounds[1]

    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        mass_total == sum(x[c] for c in components),
        C12_upper >= sum(x[c]*(data[c]['C'])/mass_total for c in components),
        C12_lower <= sum(x[c]*(data[c]['C'])/mass_total for c in components)
    ]
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    return min_cost, optimal_blend, mass_total

In [ ]:
cost,blend,mass = metal_blend(5000,[2,3],data)
print("Total cost: $" + str(cost))
print("Total mass: " + str(mass) + " kg")
print("Raw material masses:")
key1 = set(blend.keys())
for c in sorted(key1):
    print(f"{c}: {blend[c]:5.2f} kg")
key2 = set(data.keys())
C = 0
Cu = 0
Mn = 0
for c in sorted(key2):
    C += data[c]['C']*blend[c]/mass
    Cu += data[c]['Cu']*blend[c]/mass
    Mn += data[c]['Mn']*blend[c]/mass
print("Carbon content: " + str(C) + "%")
print("Copper content: " + str(Cu) + "%")
print("Manganese content: " + str(Mn) + "%")

Total cost: $5700.000000286162
Total mass: 5000 kg
Raw material masses:
A: 4000.00 kg
B:  0.00 kg
C: 1000.00 kg
D:  0.00 kg
E:  0.00 kg
F:  0.00 kg
G:  0.00 kg
Carbon content: 1.9999999999511684%
Copper content: 0.06000000472885763%
Manganese content: 1.0400000000561054%


## 5.3.2 Exercise 2. Incorporating Constraints

We'll continue this problem by incorporating all of the constraints. The constraiints include:

* Lower and upper bounds on the copper and manganese composition
* Limits on the amount of available raw material

The easiest way to proceed is to copy and paste `metal_blend` into a cell below, then add constraints one at a time until all have been incorporated.

What is the minimum price you would need to charge in $/kg to produce 5,000 kg of requested product?

In [ ]:
def metal_blend_full(mass_total, Carbon_bounds, Copper_bounds, Manganese_bounds, data):
    
    # extract bounds on carbon content
    C12_lower = Carbon_bounds[0]
    C12_upper = Carbon_bounds[1]
    Cu_lower = Copper_bounds[0]
    Cu_upper = Copper_bounds[1]
    Mn_lower = Manganese_bounds[0]
    Mn_upper = Manganese_bounds[1]

    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        mass_total == sum(x[c] for c in components),
        C12_upper >= sum(x[c]*(data[c]['C'])/mass_total for c in components),
        C12_lower <= sum(x[c]*(data[c]['C'])/mass_total for c in components),
        Cu_upper >= sum(x[c]*(data[c]['Cu'])/mass_total for c in components),
        Cu_lower <= sum(x[c]*(data[c]['Cu'])/mass_total for c in components),
        Mn_upper >= sum(x[c]*(data[c]['Mn'])/mass_total for c in components),
        Mn_lower <= sum(x[c]*(data[c]['Mn'])/mass_total for c in components),
        x['A'] <= data['A']['amount'],
        x['B'] <= data['B']['amount'],
        x['C'] <= data['C']['amount'],
        x['D'] <= data['D']['amount'],
        x['E'] <= data['E']['amount'],
        x['F'] <= data['F']['amount'],
        x['G'] <= data['G']['amount']
    ]
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    return min_cost, optimal_blend, mass_total

In [ ]:
cost,blend,mass = metal_blend_full(5000,[2,3],[0.4,0.6],[1.2,1.65],data)
print(blend)

{'A': 3999.999999424677, 'E': 27.6127214769523, 'C': 397.7630139073546, 'F': 574.6242613711523, 'D': 1.698894258951156e-07, 'B': 1.3081804390640915e-06, 'G': 2.341794167016571e-06}


In [ ]:
print("Total cost: $" + str(cost))
print("Total mass: " + str(mass) + " kg")
print("Raw material masses:")
key1 = set(blend.keys())
for c in sorted(key1):
    print(f"{c}: {blend[c]:5.2f} kg")
key2 = set(data.keys())
C = 0
Cu = 0
Mn = 0
for c in sorted(key2):
    C += data[c]['C']*blend[c]/mass
    Cu += data[c]['Cu']*blend[c]/mass
    Mn += data[c]['Mn']*blend[c]/mass
print("Carbon content: " + str(round(C,3)) + "%")
print("Copper content: " + str(round(Cu,3)) + "%")
print("Manganese content: " + str(round(Mn,3)) + "%")
rate = cost/mass
print("Minimum required unit price: $" + str(round(rate,2)) + "/kg")

Total cost: $5887.574276138117
Total mass: 5000 kg
Raw material masses:
A: 4000.00 kg
B:  0.00 kg
C: 397.76 kg
D:  0.00 kg
E: 27.61 kg
F: 574.62 kg
G:  0.00 kg
Carbon content: 2.0%
Copper content: 0.6%
Manganese content: 1.2%
Minimum required unit price: $1.18/kg


## 5.3.3 Exercise 3. Sensitivity Analysis

One of the important reasons to create optimization models is to understand the value of the raw materials that are available to you, and how to adjust operations to accomodate changing requirements. The is commonly referred to as **sensitivity analysis**.

Using the functions you've created in above exercises, answer the following questions:

1. The customer is very pleased with your initial pricing for 5,000 kg of the desired product, and now wishes to increase the order to 6,000 kg. Does your unit cost ($/kg) go up? If so, explain why your unit cost has gone up.

2. Is there an upper limit on how much product your can provide to this customer? (Use trial and error. To the nearest 100 kg, find the maximum amount of product you can ship to the customer.) What is the unit cost now?

3. After some negotiation, you and your customer settle on an order of 6,500 kg. Now you wish to negotiate with your suppliers for more raw material. How much money to you save for 1 additional kg of raw material "A"? Based on this, what price would you be willing to pay your supplier for additional "A"?



In [ ]:
cost,blend,mass = metal_blend_full(6000,[2,3],[0.4,0.6],[1.2,1.65],data)
print("Total cost: $" + str(cost))
print("Total mass: " + str(mass) + " kg")
rate = cost/mass
print("Minimum required unit price: $" + str(round(rate,2)) + "/kg")

Total cost: $7352.143776203811
Total mass: 6000 kg
Minimum required unit price: $1.23/kg


The cost of my blend increased on a per-unit mass basis because the optimization required the use of more expensive materials in order to satisfy the constraints on both composition and raw material availability.  Because the original problem, a 5000 kg order, utilized all of the raw material A, increasing the size of the order necessitates the incorporation of a more expensive material to ensure the carbon content constraint is still satisfied.

In [ ]:
cost,blend,mass = metal_blend_full(6800,[2,3],[0.4,0.6],[1.2,1.65],data)
print("Total cost: $" + str(cost))
print("Total mass: " + str(mass) + " kg")
rate = cost/mass
print("Minimum required unit price: $" + str(round(rate,2)) + "/kg")
print("Note: The optimization crashes at 6900 kg and above.")

Total cost: $8523.79937396798
Total mass: 6800 kg
Minimum required unit price: $1.25/kg
Note: The optimization crashes at 6900 kg and above.


In [ ]:
def metal_blend_sensitivity(mass_total, Carbon_bounds, Copper_bounds, Manganese_bounds, data,perturb):
    # extract bounds on carbon content
    C12_lower = Carbon_bounds[0]
    C12_upper = Carbon_bounds[1]
    Cu_lower = Copper_bounds[0]
    Cu_upper = Copper_bounds[1]
    Mn_lower = Manganese_bounds[0]
    Mn_upper = Manganese_bounds[1]
    # create set of components
    components = set(data.keys())
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    # create list of constraints
    constraints = [
        mass_total == sum(x[c] for c in components),
        C12_upper >= sum(x[c]*(data[c]['C'])/mass_total for c in components),
        C12_lower <= sum(x[c]*(data[c]['C'])/mass_total for c in components),
        Cu_upper >= sum(x[c]*(data[c]['Cu'])/mass_total for c in components),
        Cu_lower <= sum(x[c]*(data[c]['Cu'])/mass_total for c in components),
        Mn_upper >= sum(x[c]*(data[c]['Mn'])/mass_total for c in components),
        Mn_lower <= sum(x[c]*(data[c]['Mn'])/mass_total for c in components),
        x['A'] <= data['A']['amount'] + perturb,
        x['B'] <= data['B']['amount'],
        x['C'] <= data['C']['amount'],
        x['D'] <= data['D']['amount'],
        x['E'] <= data['E']['amount'],
        x['F'] <= data['F']['amount'],
        x['G'] <= data['G']['amount']
    ]
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    # return results
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    return min_cost, optimal_blend, mass_total

In [ ]:
cost0,blend0,mass0 = metal_blend_sensitivity(6500,[2,3],[0.4,0.6],[1.2,1.65],data,0)
print("Initial Cost: $" + str(round(cost0,2)))
cost1,blend1,mass1 = metal_blend_sensitivity(6500,[2,3],[0.4,0.6],[1.2,1.65],data,1)
print("With one additional unit of A: $" + str(round(cost1,2)))
cost2,blend2,mass2 = metal_blend_sensitivity(6500,[2,3],[0.4,0.6],[1.2,1.65],data,2)
print("With two additional units of A: $" + str(round(cost2,2)))
cost3,blend3,mass3 = metal_blend_sensitivity(6500,[2,3],[0.4,0.6],[1.2,1.65],data,3)
print("With 3 additional units of A: $" + str(round(cost3,2)))

Initial Cost: $8084.43
With one additional unit of A: $8084.07
With two additional units of A: $8083.71
With 3 additional units of A: $8083.35


Based on this analysis, I would be willing to pay an additional 35 cents for each EXTRA unit of A, for a total of $1.55 per unit purchased above 4000.  Of course, I would try to negotiate this extra fee down, but the maximum additional cost that still results in extra profit is 35 cents.

<!--NAVIGATION-->
< [5.2 Linear Blending Problems](https://jckantor.github.io/cbe30338-2021/05.02-Linear-Blending-Problem.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.4 Gasoline Blending](https://jckantor.github.io/cbe30338-2021/05.04-Gasoline-Blending.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.03-Homework_5.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.03-Homework_5.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>